In [53]:
import json
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [54]:
df_train = pd.read_csv('first_round_training_data.csv')
df_test = pd.read_csv('first_round_testing_data.csv')

In [55]:
dit = {'Excellent':0,'Good':1,'Pass':2,'Fail':3}
df_train['label'] = df_train['Quality_label'].map(dit)
# 将特征转换成数字

In [56]:
df_train=df_train.drop(labels=['Attribute1','Attribute2','Attribute3','Attribute4','Attribute5','Attribute6','Attribute7',
                       'Attribute8','Attribute9','Attribute10','Quality_label', ],axis=1)
# 还有人说删除前四个特征，但是删除之后得分反而变回0.4了

In [57]:
data=df_train.drop(labels='label',axis=1)
target = df_train['label']
X_train,X_test,y_train,y_test =train_test_split(data,target,test_size=0.2)


In [68]:

# 创建成lgb特征的数据集格式
lgb_train = lgb.Dataset(X_train, y_train) # 将数据保存到LightGBM二进制文件将使加载更快
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)  # 创建验证数据
 
# 将参数写成字典下形式
params = {  
    'boosting_type': 'gbdt',  
    'objective': 'multiclass',  
    'num_class': 4,  
    'metric': 'multi_error',  
    'num_leaves': 120,  
    'min_data_in_leaf': 100,  
    'learning_rate': 0.06,  
    'feature_fraction': 0.8,  
    'bagging_fraction': 0.8,  
    'bagging_freq': 5,  
    'lambda_l1': 0.4,  
    'lambda_l2': 0.5,  
    'min_gain_to_split': 0.2,  
    'verbose': -1, 
}
print('Start training...')
# 训练 cv and train
gbm = lgb.train(params,lgb_train,num_boost_round=200,valid_sets=lgb_eval,early_stopping_rounds=100) # 训练数据需要参数列表和数据集
 


Start training...
[1]	valid_0's multi_error: 0.618333
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's multi_error: 0.618333
[3]	valid_0's multi_error: 0.618333
[4]	valid_0's multi_error: 0.618333
[5]	valid_0's multi_error: 0.618333
[6]	valid_0's multi_error: 0.618333
[7]	valid_0's multi_error: 0.618333
[8]	valid_0's multi_error: 0.6175
[9]	valid_0's multi_error: 0.611667
[10]	valid_0's multi_error: 0.609167
[11]	valid_0's multi_error: 0.595
[12]	valid_0's multi_error: 0.585
[13]	valid_0's multi_error: 0.573333
[14]	valid_0's multi_error: 0.561667
[15]	valid_0's multi_error: 0.551667
[16]	valid_0's multi_error: 0.544167
[17]	valid_0's multi_error: 0.538333
[18]	valid_0's multi_error: 0.534167
[19]	valid_0's multi_error: 0.535833
[20]	valid_0's multi_error: 0.533333
[21]	valid_0's multi_error: 0.5325
[22]	valid_0's multi_error: 0.525
[23]	valid_0's multi_error: 0.5175
[24]	valid_0's multi_error: 0.5175
[25]	valid_0's multi_error: 0.518333
[26]	valid_0's multi

In [59]:

# y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration) #如果在训练期间启用了早期停止，可以通过best_iteration方式从最佳迭代中获得预测
# # 评估模型
# print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5) # 计算真实值和预测值之间的均方根误差


In [69]:
# df_test

,Group,Parameter1,Parameter10,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9
0,0,0.235279,0.195680,0.351488,0.186814,8.645558e-02,1.434060,1.230409,0.038483,2.931083,2.005852e+00
1,0,0.043799,0.523976,32.590688,0.886285,2.970249e+01,0.027434,0.016852,0.600827,0.118077,8.876113e+02
2,0,21.466737,0.523976,11.847792,79499.554454,1.354461e+04,0.027434,0.016852,0.600827,0.118077,8.876113e+02
3,0,0.703795,0.010192,0.018080,317.015599,3.808756e+01,0.428092,0.314162,0.038483,17.850021,5.184951e-02
4,0,10.447960,0.010192,15.983891,0.996049,1.671569e+01,0.477823,0.464037,0.038483,17.850021,5.184951e-02
5,0,733.439920,0.010192,99.698641,3.202776,3.136804e+01,0.477823,0.464037,0.038483,17.850021,5.184951e-02
6,0,0.149962,0.010192,2.910066,2.187656,3.317446e+02,0.533331,0.464037,0.038483,17.850021,5.184951e-02
7,0,0.060635,0.010192,0.016492,0.007373,7.183437e+03,0.533331,0.464037,0.038483,17.850021,5.184951e-02
8,0,971.284623,0.010192,29.954312,3.639672,1.363832e-01,0.477823,0.381815,0.038483,17.850021,5.184951e-02
9,0,1372.664549,0.010192,2.103451,0.531944,5.802723e+00,0.477823,0.464037,0.038483,17.850021,5.184951e-02


In [70]:
example = pd.read_csv('submit_example.csv')

In [71]:
prediction=gbm.predict(df_test.drop(labels='Group',axis=1))
# prediction

In [72]:
prediction.shape

(6000, 4)

In [78]:
prediction

array([[0.1183566 , 0.25202043, 0.34611418, 0.28350879],
       [0.08043937, 0.36372007, 0.3622726 , 0.19356796],
       [0.09974858, 0.34488439, 0.33149341, 0.22387361],
       ...,
       [0.07682772, 0.20839072, 0.14712284, 0.56765872],
       [0.06447932, 0.21081525, 0.38391317, 0.34079226],
       [0.12884221, 0.21303179, 0.5843381 , 0.0737879 ]])

In [74]:
df_test.shape

(6000, 11)

In [80]:
prediction[:,0]

array([0.1183566 , 0.08043937, 0.09974858, ..., 0.07682772, 0.06447932,
       0.12884221])

In [81]:
df_test['Excellent ratio'] = prediction[:,0]
df_test['Good ratio'] = prediction[:,1]
df_test['Pass ratio'] = prediction[:,2]
df_test['Fail ratio'] = prediction[:,3]
result = pd.DataFrame({'Group':example.Group})

In [82]:
df_test.head()

,Group,Parameter1,Parameter10,Parameter2,Parameter3,Parameter4,Parameter5,Parameter6,Parameter7,Parameter8,Parameter9,Excellent ratio,Good ratio,Pass ratio,Fail ratio
0,0,0.235279,0.195680,0.351488,0.186814,0.086456,1.434060,1.230409,0.038483,2.931083,2.005852,0.118357,0.252020,0.346114,0.283509
1,0,0.043799,0.523976,32.590688,0.886285,29.702490,0.027434,0.016852,0.600827,0.118077,887.611257,0.080439,0.363720,0.362273,0.193568
2,0,21.466737,0.523976,11.847792,79499.554454,13544.605343,0.027434,0.016852,0.600827,0.118077,887.611257,0.099749,0.344884,0.331493,0.223874
3,0,0.703795,0.010192,0.018080,317.015599,38.087562,0.428092,0.314162,0.038483,17.850021,0.051850,0.097607,0.485367,0.156690,0.260336
4,0,10.447960,0.010192,15.983891,0.996049,16.715690,0.477823,0.464037,0.038483,17.850021,0.051850,0.145350,0.388697,0.194844,0.271109


In [90]:
for group_num in range(120):
#     print(group_num)
#     all_counts = df_test.loc[df_test['Group'] ==group_num]['Excellent ratio'].mean()
#     print(all_counts)
    try:
        result.loc[result['Group'] ==group_num,'Excellent ratio'] = df_test.loc[df_test['Group'] ==group_num]['Excellent ratio'].mean()
    except Exception as e:
        result.loc[result['Group'] ==group_num,'Excellent ratio'] = 0
    try:
        result.loc[result['Group'] ==group_num,'Good ratio'] = df_test.loc[df_test['Group'] ==group_num]['Good ratio'].mean()
    except Exception as e:
        result.loc[result['Group'] ==group_num,'Good ratio'] = 0
    try:
        result.loc[result['Group'] ==group_num,'Pass ratio'] = df_test.loc[df_test['Group'] ==group_num]['Pass ratio'].mean()
    except Exception as e:
        result.loc[result['Group'] ==group_num,'Pass ratio'] = 0
    try:
        result.loc[result['Group'] ==group_num,'Fail ratio'] = df_test.loc[df_test['Group'] ==group_num]['Fail ratio'].mean()
    except Exception as e:
        result.loc[result['Group'] ==group_num,'Fail ratio'] = 0

In [91]:
result.to_csv(r'result.csv',index=False)